In [ ]:
import os
#os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import pandas as pd
import numpy as np
import math, os, random, json, pickle
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from copy import deepcopy

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
# from cmapPy.pandasGEXpress.parse import parse
from rdkit import Chem
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem.Fingerprints import FingerprintMols

import torch
import shutil

import os
import csv
from tqdm import tqdm
import shutil

from torchdrug import data, utils
from torchdrug.core import Registry as R

from madrigal.utils import DATA_DIR, BASE_DIR
 

02:59:49   Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.


In [ ]:
def remove_similar(df, name, root):
    
    smiles = df.smiles.tolist()
    
    test_between = pd.read_csv(DATA_DIR+'polypharmacy/DrugBank/split_by_drugs_random/test_between_df.csv')
    test_within = pd.read_csv(DATA_DIR+'polypharmacy/DrugBank/split_by_drugs_random/test_within_df.csv')
    all_smiles = pd.read_csv(DATA_DIR+'views_features/combined_metadata_reindexed_ddi.csv')
    all_smiles = all_smiles.canonical_smiles
    test = np.concatenate((test_between['head'].values, test_within['head'].values, test_within['tail'].values ))
    test = np.unique(test)
    test_smiles = all_smiles.loc[test].values
    
    def get_fingerprints(l):
        all_mols = [AllChem.AddHs(Chem.MolFromSmiles(sm)) for sm in tqdm(l)] 
        all_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=256) for mol in tqdm(all_mols)]
        return all_fps

    fps = get_fingerprints(smiles)
    test_fps = get_fingerprints(test_smiles)
    
    scores = []
    for n in tqdm(fps):
        score = DataStructs.BulkTanimotoSimilarity(n, test_fps)
        scores.append(score)
        
    scores = np.array(scores)
    msk = np.where(scores >= 0.98)
    to_remove = msk[0]
    
    for i, smile in enumerate(smiles):
        try:
            x = Chem.MolFromSmiles(smile)
        except:
            print(f'Removed {i}')
    
    bad_df = df.index.isin(to_remove)
    df = df[~bad_df]
    
    df.to_csv(os.path.join(root, f'{name}_filtered_torchdrug_new.csv'), index=False)
    
    
    small = df.sample(n=100, replace=False, random_state=1)
    small.to_csv(os.path.join(root, f'{name}_filtered_torchdrug_new_small.csv'))
    

In [ ]:
path = './molecule-datasets/'
url = "http://snap.stanford.edu/gnn-pretrain/data/chem_dataset.zip"
md5 = "e95da4dffa0fdb1d4af2726bdf8c23e0"
member = "dataset/zinc_standard_agent/processed/smiles.csv"
zip_file_name = utils.download(url, path, md5=md5)
save_file = utils.extract(zip_file=zip_file_name, member=member)
neo_save_file = os.path.join(os.path.dirname(zip_file_name), 'zinc2m_'+os.path.basename(member))
shutil.move(save_file, neo_save_file)

In [18]:
df = pd.read_csv(neo_save_file, header=None)
df.columns = ['smiles']

In [ ]:
remove_similar(df, 'zinc2m_test', path)

# Training

In [1]:
import torch
from torchdrug import data, datasets
import os
from torchdrug import core, models, tasks, utils
from torchdrug import data, utils
from torchdrug.core import Registry as R
import os
import sys
import logging
from itertools import islice
import argparse

import torch
from torch import distributed as dist
from torch import nn
from torch.utils import data as torch_data

from rdkit import Chem
import numpy as np

class ZINC_filt(data.MoleculeDataset):

    def __init__(self, path, verbose=1, **kwargs):
        
        with open(path, "r") as fin:
            reader = csv.reader(fin)
            if verbose:
                reader = iter(tqdm(reader, "Loading %s" % path, utils.get_line_count(neo_save_file)))
            smiles_list = []

            for idx, values in enumerate(reader):
                smiles = values[0]
                smiles_list.append(smiles)
                

        targets = {}
        self.load_smiles(smiles_list, targets, lazy=True, verbose=verbose, **kwargs)


In [ ]:
dataset = ZINC_filt(os.path.join(path, 'zinc2m_filtered_torchdrug_new.csv'))

In [45]:
gin_model = models.GraphIsomorphismNetwork(input_dim=67, 
                                       hidden_dims=[128, 128, 128]+[128],
                                       edge_input_dim=18, 
                                       num_mlp_layer=3, 
                                       eps=0, 
                                       batch_norm=False, 
                                       activation='relu', 
                                       readout='mean')


task = tasks.AttributeMasking(gin_model, mask_rate=0.15)

optimizer = torch.optim.Adam(task.parameters(), lr=1e-3)
solver = core.Engine(task, dataset, None, None, optimizer, gpus=[0], batch_size=10000)

solver.train(num_epoch=100)

15:59:44   Preprocess training set
15:59:44   {'batch_size': 10000,
 'class': 'core.Engine',
 'gpus': [0],
 'gradient_interval': 1,
 'log_interval': 100,
 'logger': 'logging',
 'num_worker': 0,
 'optimizer': {'amsgrad': False,
               'betas': (0.9, 0.999),
               'capturable': False,
               'class': 'optim.Adam',
               'differentiable': False,
               'eps': 1e-08,
               'foreach': None,
               'fused': False,
               'lr': 0.001,
               'maximize': False,
               'weight_decay': 0},
 'scheduler': None,
 'task': {'class': 'tasks.AttributeMasking',
          'graph_construction_model': None,
          'mask_rate': 0.15,
          'model': {'activation': 'relu',
                    'batch_norm': False,
                    'class': 'models.GIN',
                    'concat_hidden': False,
                    'edge_input_dim': 18,
                    'eps': 0,
                    'hidden_dims': [128, 128, 128, 1

15:59:51   Epoch 17 end
15:59:51   duration: 0.36 secs
15:59:51   speed: 2.78 batch / sec
15:59:51   ETA: 35.08 secs
15:59:51   max GPU memory: 48.6 MiB
15:59:51   ------------------------------
15:59:51   average accuracy: 0.722892
15:59:51   average cross entropy: 0.942898
15:59:51   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
15:59:51   Epoch 18 begin
15:59:52   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
15:59:52   Epoch 18 end
15:59:52   duration: 0.35 secs
15:59:52   speed: 2.84 batch / sec
15:59:52   ETA: 34.33 secs
15:59:52   max GPU memory: 48.6 MiB
15:59:52   ------------------------------
15:59:52   average accuracy: 0.692771
15:59:52   average cross entropy: 1.26135
15:59:52   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
15:59:52   Epoch 19 begin
15:59:52   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
15:59:52   Epoch 19 end
15:59:52   duration: 0.34 secs
15:59:52   speed: 2.93 batch / sec
15:59:52   ETA: 33.58 secs
15:59:52   max GPU memory: 48.6 MiB
15:59:52   ------------------------------
15:59:52   average accuracy: 0.710

15:59:58   max GPU memory: 45.7 MiB
15:59:58   ------------------------------
15:59:58   average accuracy: 0.771084
15:59:58   average cross entropy: 0.737255
15:59:58   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
15:59:58   Epoch 39 begin
15:59:59   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
15:59:59   Epoch 39 end
15:59:59   duration: 0.35 secs
15:59:59   speed: 2.84 batch / sec
15:59:59   ETA: 22.74 secs
15:59:59   max GPU memory: 45.7 MiB
15:59:59   ------------------------------
15:59:59   average accuracy: 0.801205
15:59:59   average cross entropy: 0.690466
15:59:59   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
15:59:59   Epoch 40 begin
15:59:59   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
15:59:59   Epoch 40 end
15:59:59   duration: 0.36 secs
15:59:59   speed: 2.79 batch / sec
15:59:59   ETA: 22.33 secs
15:59:59   max GPU memory: 45.7 MiB
15:59:59   ------------------------------
15:59:59   average accuracy: 0.777108
15:59:59   average cross entropy: 0.725898
15:59:59   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
15:59:59   Epoch 41 begin
1

16:00:05   average cross entropy: 0.582226
16:00:05   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:00:05   Epoch 60 begin
16:00:06   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:00:06   Epoch 60 end
16:00:06   duration: 0.34 secs
16:00:06   speed: 2.92 batch / sec
16:00:06   ETA: 14.13 secs
16:00:06   max GPU memory: 45.7 MiB
16:00:06   ------------------------------
16:00:06   average accuracy: 0.76506
16:00:06   average cross entropy: 0.630001
16:00:06   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:00:06   Epoch 61 begin
16:00:06   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:00:06   Epoch 61 end
16:00:06   duration: 0.36 secs
16:00:06   speed: 2.82 batch / sec
16:00:06   ETA: 13.77 secs
16:00:06   max GPU memory: 45.7 MiB
16:00:06   ------------------------------
16:00:06   average accuracy: 0.740964
16:00:06   average cross entropy: 0.685635
16:00:06   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:00:06   Epoch 62 begin
16:00:06   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:00:06   Epoch 62 end
16:00:06   duration: 0.35 secs
16:00:06   speed: 2.8

16:00:13   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:00:13   Epoch 81 end
16:00:13   duration: 0.34 secs
16:00:13   speed: 2.92 batch / sec
16:00:13   ETA: 6.46 secs
16:00:13   max GPU memory: 45.7 MiB
16:00:13   ------------------------------
16:00:13   average accuracy: 0.810241
16:00:13   average cross entropy: 0.623193
16:00:13   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:00:13   Epoch 82 begin
16:00:13   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:00:13   Epoch 82 end
16:00:13   duration: 0.35 secs
16:00:13   speed: 2.89 batch / sec
16:00:13   ETA: 6.10 secs
16:00:13   max GPU memory: 45.7 MiB
16:00:13   ------------------------------
16:00:13   average accuracy: 0.85241
16:00:13   average cross entropy: 0.490624
16:00:13   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:00:13   Epoch 83 begin
16:00:14   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:00:14   Epoch 83 end
16:00:14   duration: 0.35 secs
16:00:14   speed: 2.88 batch / sec
16:00:14   ETA: 5.74 secs
16:00:14   max GPU memory: 45.7 MiB
16:00:14   --------------------------

In [6]:
model = models.GraphIsomorphismNetwork(input_dim=67, 
                                       hidden_dims=[128, 128, 128]+[128],
                                       edge_input_dim=18, 
                                       num_mlp_layer=3, 
                                       eps=0, 
                                       batch_norm=False, 
                                       activation='relu', 
                                       readout='mean')
                   
task = tasks.PropertyPrediction(model, task=dataset.tasks, criterion="bce", metric=("auprc", "auroc"))
                                
optimizer = torch.optim.Adam(task.parameters(), lr=1e-3)
solver = core.Engine(task, train_set, valid_set, test_set, optimizer, gpus=[0], batch_size=100)
solver.train(num_epoch=20)
solver.evaluate("valid")

In [7]:
check = torch.load('GIN_256x4_muv.pt', map_location=torch.device('cuda'))

In [8]:
model.load_state_dict(check)

<All keys matched successfully>

In [ ]:
import pickle

fl = BASE_DIR + "raw_data/drugbank/raw_drugbank_df_text_reference_filtered.pkl"

objects = []
with (open(fl, "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

In [10]:
df = objects[0]
df = df[~df.smiles.isna()]
smiles = df.smiles
#smiles.to_csv('smiles.csv', index=None)
smiles = smiles.tolist()

In [ ]:
import pickle

fl = BASE_DIR + "raw_data/drugbank/processed/drugbank_info.pkl"

objects = []
with (open(fl, "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

In [13]:
prop = objects[0]
prop['prop_str'] = [','.join(map(str, l)) if None not in l else pd.NA for l in prop['classifications']]
prop['atc'] = [l[-1] if len(l)>0 else pd.NA for l in prop['atc_codes']]

In [14]:
smiles_new = []
valid = []
for i,smile in enumerate(smiles):
    good = True
    try:
        m = Chem.MolFromSmiles(smile)
        for atom in m.GetAtoms():
            if atom.GetFormalCharge() >= 6:
                good = False
        if good:
            smiles_new.append(Chem.MolToSmiles(m))
            valid.append(i)
    except:
        continue

[15:08:09] Explicit valence for atom # 0 N, 4, is greater than permitted
[15:08:09] Explicit valence for atom # 0 N, 4, is greater than permitted
[15:08:09] Explicit valence for atom # 0 N, 4, is greater than permitted
[15:08:10] Explicit valence for atom # 13 Cl, 5, is greater than permitted
[15:08:10] SMILES Parse Error: syntax error while parsing: OS(O)(O)C1=CC=C(C=C1)C-1=C2\C=CC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC=C(C=C1)S(O)(O)O)C1=CC=C(C=C1)S([O-])([O-])[O-])\C1=CC=C(C=C1)S(O)(O)[O-]
[15:08:10] SMILES Parse Error: Failed parsing SMILES 'OS(O)(O)C1=CC=C(C=C1)C-1=C2\C=CC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC=C(C=C1)S(O)(O)O)C1=CC=C(C=C1)S([O-])([O-])[O-])\C1=CC=C(C=C1)S(O)(O)[O-]' for input: 'OS(O)(O)C1=CC=C(C=C1)C-1=C2\C=CC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC=C(C=C1)S(O)(O)O)C1=CC=C(C=C1)S([O-])([O-])[O-])\C1=CC=C(C=C1)S(O)(O)[O-]'
[15:08:10] Explicit valence for atom # 19 O, 3, is greater than permitted
[15:

In [15]:
df = df.iloc[valid,:]
ids = df.drugbank_id

In [16]:
prop_fn = pd.merge(prop[["drugbank_id", "prop_str", 'atc']], df[["drugbank_id", "smiles"]], on="drugbank_id", how ='outer')

In [17]:
prop_fn = prop_fn[prop_fn.drugbank_id.isin(ids)]

In [18]:
prop_fn.to_csv('smiles_property.csv')

In [23]:
class MUV_filt_smiles(data.MoleculeDataset):
    """
    Subset of PubChem BioAssay by applying a refined nearest neighbor analysis.

    Statistics:
        - #Molecule: 93,087
        - #Classification task: 17

    Parameters:
        path (str): path to store the dataset
        verbose (int, optional): output verbose level
        **kwargs
    """

    target_fields = ["MUV-466", "MUV-548", "MUV-600", "MUV-644", "MUV-652", "MUV-689", "MUV-692", "MUV-712", "MUV-713",
                     "MUV-733", "MUV-737", "MUV-810", "MUV-832", "MUV-846", "MUV-852", "MUV-858", "MUV-859"]

    def __init__(self, smiles, verbose=1, **kwargs):

        self.load_smiles(smiles, targets={'dummy': [0]*len(smiles)}, verbose=verbose, **kwargs)

In [24]:
dataset = MUV_filt_smiles(smiles_new)

Constructing molecules from SMILES:   0%|                                                      | 0/11563 [00:00<?, ?it/s]/home/vau974/.local/lib/python3.9/site-packages/torchdrug/data/feature.py:42: UserWarning: Unknown value `Co`
  warnings.warn("Unknown value `%s`" % x)
Constructing molecules from SMILES:   1%|▍                                           | 99/11563 [00:00<00:32, 349.03it/s]/home/vau974/.local/lib/python3.9/site-packages/torchdrug/data/feature.py:42: UserWarning: Unknown value `Gd`
  warnings.warn("Unknown value `%s`" % x)
Constructing molecules from SMILES:   1%|▌                                          | 137/11563 [00:00<00:32, 356.85it/s]/home/vau974/.local/lib/python3.9/site-packages/torchdrug/data/feature.py:42: UserWarning: Unknown value `Ca`
  warnings.warn("Unknown value `%s`" % x)
Constructing molecules from SMILES:   2%|▊                                          | 215/11563 [00:00<00:32, 350.36it/s]/home/vau974/.local/lib/python3.9/site-packages/torchdrug/da

In [25]:
samples = []
categories = set()
for sample in dataset:
    samples.append(sample)
samples = data.graph_collate(samples)
samples = utils.cuda(samples)
#preds = F.sigmoid(task.predict(samples))
#targets = task.target(samples)
model = model.to('cuda')

In [ ]:
emb = model(samples['graph'], samples['graph'].node_feature.float())['graph_feature']

In [27]:
emb

tensor([[8.8344e+01, 6.5148e+01, 1.1598e+02,  ..., 0.0000e+00, 1.8206e+01,
         9.0949e+01],
        [9.2018e+01, 6.3415e+01, 1.2655e+02,  ..., 1.8414e-03, 1.7941e+01,
         9.7125e+01],
        [9.3063e+01, 6.6494e+01, 1.2679e+02,  ..., 1.7605e-03, 1.8900e+01,
         9.8423e+01],
        ...,
        [1.0383e+02, 7.2722e+01, 1.5237e+02,  ..., 0.0000e+00, 2.1587e+01,
         1.3406e+02],
        [1.1406e+02, 8.2688e+01, 1.5316e+02,  ..., 0.0000e+00, 2.7641e+01,
         1.4394e+02],
        [1.1814e+00, 9.8479e-01, 1.2135e+00,  ..., 0.0000e+00, 3.3273e-01,
         7.9385e-01]], device='cuda:0', grad_fn=<DivBackward0>)

In [28]:
np.save('structure_embs_muv.npy', emb.detach().cpu().numpy())